#Integrantes de la tarea

Integrantes del grupo:

* Integrante 1:Matias Rodriguez U. Rut: 18362815-1
* Integrante 2:Paula Canales G. Rut:18845155-1
* Integrante 3:Sebastián Alday Rut:18294095-k
* Integrante 4:Álvaro Neira R. Rut:13757209-5

# Importación de Librerias

Estas son las bibliotecas y módulos necesarios.

In [ ]:
import pandas as pd
import re

import spacy
import nltk
import sklearn
import wordcloud
import matplotlib.pyplot as plt
import pandas as pd
import random

# Datos Necesarios

Descargamos los datos necesarios para las actividades

In [ ]:
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/aysen.csv
#!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/aysen_corto.csv
#!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/proveedoresA.txt
#!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/other_vhosts_access.log
nltk.download('stopwords')
nltk.download('wordnet')
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp_es = es_core_news_sm.load()

--2021-12-03 03:02:34--  https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/aysen.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29482924 (28M) [text/plain]
Saving to: ‘aysen.csv’

aysen.csv           100%[===================>]  28.12M   102MB/s    in 0.3s    

2021-12-03 03:02:35 (102 MB/s) - ‘aysen.csv’ saved [29482924/29482924]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 16.2 MB 2.4 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=335f6daddc0286a4b206112f4a02ecfb

# 1 Limpieza de la base de datos 


##Dataset

In [ ]:
aysen = pd.read_csv("aysen.csv", sep=";", na_values=["Otro",""])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 3. Análisis no supervisado

**En esta parte volví a calcular el D10K porque creo que así es más mejor**


In [ ]:
aysen_data = aysen[["SOSPECHA_DIAG","PRESTA_EST"]]

In [ ]:
import json
rank = json.loads(aysen_data.PRESTA_EST.value_counts().to_json())
rank = pd.DataFrame(list(rank.items()),columns = ['PRESTA_EST','PRESTA_EST_COUNT'])
rank = rank.loc[rank['PRESTA_EST_COUNT'] > 10000]
rank

,PRESTA_EST,PRESTA_EST_COUNT
0,TRAUMATOLOGIA,24004
1,OFTALMOLOGIA,21482
2,CIRUGIA ADULTO,17321
3,OTORRINOLARINGOLOGIA,13663
4,NEUROLOGIA,12316
5,MEDICINA INTERNA,11408
6,GINECOLOGIA,10871
7,ENDODONCIA,10225


In [ ]:
rank_list = rank.PRESTA_EST.values.tolist()
rank_list

['TRAUMATOLOGIA',
 'OFTALMOLOGIA',
 'CIRUGIA ADULTO',
 'OTORRINOLARINGOLOGIA',
 'NEUROLOGIA',
 'MEDICINA INTERNA',
 'GINECOLOGIA',
 'ENDODONCIA']

In [ ]:
D10K = aysen_data.loc[aysen_data['PRESTA_EST'].isin(rank_list)]
D10K

,SOSPECHA_DIAG,PRESTA_EST
0,Celulitis y absceso de boca,ENDODONCIA
4,Celulitis y absceso de boca,ENDODONCIA
6,FISURA ANAL,CIRUGIA ADULTO
7,Periodontitis apical aguda originada en la pulpa,ENDODONCIA
8,"Osteomielitis, no especificada",TRAUMATOLOGIA
...,...,...
217886,HIPERTENSION ESENCIAL (PRIMARIA),MEDICINA INTERNA
217888,Consulta no Especificada,MEDICINA INTERNA
217889,HERNIAS DEL NUCLEO PULPOSO,NEUROLOGIA
217892,Consulta no Especificada,CIRUGIA ADULTO


## 3.1 Probabilidad de cada categoría en D10K

In [ ]:
total = rank.PRESTA_EST_COUNT.sum()
rank['PROB'] = rank.PRESTA_EST_COUNT/total
rank

,PRESTA_EST,PRESTA_EST_COUNT,PROB
0,TRAUMATOLOGIA,24004,0.197906
1,OFTALMOLOGIA,21482,0.177113
2,CIRUGIA ADULTO,17321,0.142806
3,OTORRINOLARINGOLOGIA,13663,0.112647
4,NEUROLOGIA,12316,0.101542
5,MEDICINA INTERNA,11408,0.094056
6,GINECOLOGIA,10871,0.089628
7,ENDODONCIA,10225,0.084302


## 3.2 Diccionario P(palabra|categoria).

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(D10K, test_size=0.1)

In [ ]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer() #Se podrían quitar las stopwords
term_doc_matrix_train = count_vectorizer.fit_transform(train.SOSPECHA_DIAG)
term_doc_matrix_train

<109161x4348 sparse matrix of type '<class 'numpy.int64'>'
	with 444158 stored elements in Compressed Sparse Row format>

In [ ]:
import numpy as np

p_word_category = {}
for index, row in rank.iterrows():
  category = row['PRESTA_EST']
  count_c = row['PRESTA_EST_COUNT']
  # Se obtienen las apariciones de cada una de las palabras en cada clase
  c_w = np.asarray(term_doc_matrix_train.todense()[train.PRESTA_EST == category,:].sum(0)).reshape(-1)
  # Se calcula la probabilidad
  p_w = (c_w + 1) / count_c
  p_word_category[category] = p_w
p_word_category

{'CIRUGIA ADULTO': array([1.73200162e-04, 5.77333872e-05, 5.77333872e-05, ...,
        5.77333872e-05, 5.77333872e-05, 5.77333872e-05]),
 'ENDODONCIA': array([9.7799511e-05, 9.7799511e-05, 9.7799511e-05, ..., 9.7799511e-05,
        9.7799511e-05, 9.7799511e-05]),
 'GINECOLOGIA': array([9.19878576e-05, 9.19878576e-05, 9.19878576e-05, ...,
        4.59939288e-04, 1.15904701e-02, 9.19878576e-05]),
 'MEDICINA INTERNA': array([8.76577840e-05, 2.62973352e-04, 8.76577840e-05, ...,
        8.76577840e-05, 3.50631136e-04, 8.76577840e-05]),
 'NEUROLOGIA': array([8.11951932e-05, 8.11951932e-05, 8.11951932e-05, ...,
        8.11951932e-05, 8.11951932e-05, 8.11951932e-05]),
 'OFTALMOLOGIA': array([4.65506005e-05, 4.65506005e-05, 4.65506005e-05, ...,
        4.65506005e-05, 4.65506005e-05, 4.65506005e-05]),
 'OTORRINOLARINGOLOGIA': array([7.31903681e-05, 7.31903681e-05, 1.46380736e-04, ...,
        7.31903681e-05, 1.46380736e-04, 1.46380736e-04]),
 'TRAUMATOLOGIA': array([4.16597234e-05, 4.16597234e